Design a recursion to produce plot in Figure 6. Hint: using a recursive function, produce
vectors px, py with respectively x and y coordinates of endpoints of
your polygons, separating polygons by NaN values

# Recursive Way

In [ ]:
using BenchmarkTools

function sierpinski(x,y,size)
    if size == 1
        return ([x,x+1,x,NaN], [y,y,y+1,NaN])
    end

    top_x, top_y = sierpinski(x,y+(2^size), size-1)
    right_x, right_y = sierpinski(x+(2^size),y, size-1)
    base_x, base_y = sierpinski(x,y, size-1)

    return ([base_x;top_x;right_x], 
            [base_y;top_y; right_y])
end


px, py = @btime sierpinski(0,0,11)
using Plots;
gr();
plot(px,py,seriestype=:shape,size=(500,500),
linewidth=1,grid=false,axis=false,label=false)

([0.0, 1.0, 0.0, NaN, 0.0, 1.0, 0.0, NaN, 4.0, 5.0  …  4088.0, NaN, 4088.0, 4089.0, 4088.0, NaN, 4092.0, 4093.0, 4092.0, NaN], [0.0, 0.0, 1.0, NaN, 4.0, 4.0, 5.0, NaN, 0.0, 0.0  …  1.0, NaN, 4.0, 4.0, 5.0, NaN, 0.0, 0.0, 1.0, NaN])

# Shift Based

In [ ]:
function sierpinski_shift(x,y,size)
    if size == 1
        return ([0,1,0,NaN] .+ x, [0,0,1,NaN] .+ y)
    end

    base_x, base_y = sierpinski_shift(x,y, size-1)
    top_y = base_y .+ 2^size
    right_x = base_x .+ 2^size

    return ([base_x;base_x;right_x], 
            [base_y;top_y;base_y])
end


px, py = @btime sierpinski_shift(0,0,11)
using Plots;
gr();
plot(px,py,seriestype=:shape,size=(500,500),
linewidth=1,grid=false,axis=false,label=false)

([0.0, 1.0, 0.0, NaN, 0.0, 1.0, 0.0, NaN, 4.0, 5.0  …  4088.0, NaN, 4088.0, 4089.0, 4088.0, NaN, 4092.0, 4093.0, 4092.0, NaN], [0.0, 0.0, 1.0, NaN, 4.0, 4.0, 5.0, NaN, 0.0, 0.0  …  1.0, NaN, 4.0, 4.0, 5.0, NaN, 0.0, 0.0, 1.0, NaN])

## Comments
25x improvements wrt to the previous implementation

# Shift + Memo

In [ ]:
using BenchmarkTools

cache = [
    ([0,1,0,NaN], [0,0,1,NaN])
]
function sierpinski_sm(x,y,size)
    if x == 0 && y == 0
        print("calling with size $(size)")
    end
    base_x = []
    base_y = []
    if size-1 <= length(cache)
        px, py = cache[size-1]
        base_x = px .+ x
        base_y = py .+ y
    else
        base_x, base_y = sierpinski_sm(0,0, size-1)
        push!(cache, (base_x, base_y))
        base_x = base_x .+ x
        base_y = base_y .+ y
    end

    return ([base_x;base_x;base_x .+ 2^(size-2)], 
            [base_y;base_y .+ 2^(size-2);base_y])
end

px, py = @btime sierpinski_sm(0,0,11)

([0.0, 1.0, 0.0, NaN, 0.0, 1.0, 0.0, NaN, 1.0, 2.0  …  1022.0, NaN, 1022.0, 1023.0, 1022.0, NaN, 1023.0, 1024.0, 1023.0, NaN], [0.0, 0.0, 1.0, NaN, 1.0, 1.0, 2.0, NaN, 0.0, 0.0  …  1.0, NaN, 1.0, 1.0, 2.0, NaN, 0.0, 0.0, 1.0, NaN])